In [ ]:
# default_exp loss

# Loss

In [ ]:
#hide
from fastcore.test import *
from nbdev.showdoc import *

In [ ]:
#export
from fastai.basics import *

In [ ]:
#export

class CombinedLoss():
    """Applies loss functions to multiple model outputs and sums them. 
    If applicable, it can decode and compute activations for each model output."""

    def __init__(self, *loss_funcs, weight=None):
        if weight is None:
            weight = [1.]*len(loss_funcs)
        self.weight = weight
        self.loss_funcs = loss_funcs
    
    def __call__(self, outs, *targets, **kwargs):
        return sum([
            w*loss_func(out, target) 
            for loss_func, w, out, target in zip(self.loss_funcs, self.weight, outs, targets)
        ])
    
    def activation(self, outs): 
        return [getattr(loss_func, 'activation', noop)(out) for loss_func, out in zip(self.loss_funcs, outs)]
    
    def decodes(self, outs):
        return [getattr(loss_func, 'decodes', noop)(out) for loss_func, out in zip(self.loss_funcs, outs)]
    

Assume that a multi-task learning model produces two outputs:
1. The logits for multi-class single-label classification, for which we want to use cross-entropy loss and softmax activation
2. A logit for single-class classification, for which we want to use binary cross-entropy and sigmoid activation

`CombinedLoss` enables using the corresponding loss function and its activation function for each model output.

In [ ]:
from fastai.vision.all import *

ce = CrossEntropyLossFlat()
bce = BCEWithLogitsLossFlat()
comb_loss = CombinedLoss(ce, bce)

bs = 8
target1, output1 = torch.randint(0, 5, (bs,)), torch.randn(bs, 5) # 5 classes
target2, output2 = torch.randint(0, 2, (bs,), dtype=float), torch.randn(bs)*10
actual = comb_loss((output1, output2), target1, target2)

loss1 = ce(output1, target1)
loss2 = bce(output2, target2)
expected = loss1 + loss2
test_close(expected, actual)

# activations
actual_acts_output1, actual_acts_output2 = comb_loss.activation([output1, output2])
expected_acts_output1, expected_acts_output2 = ce.activation(output1), bce.activation(output2)
test_close(expected_acts_output1, actual_acts_output1)
test_eq(expected_acts_output2, actual_acts_output2)

# decoding
actual_decoded_output1, actual_decoded_output2 = comb_loss.decodes([output1, output2])
expected_decoded_output1, expected_decoded_output2 = ce.decodes(output1), bce.decodes(output2)
test_close(expected_decoded_output1, actual_decoded_output1)
test_eq(expected_decoded_output2, actual_decoded_output2)


Here are raw model outputs (logits):

In [ ]:
[output1, output2]

[tensor([[-0.0704,  0.3978, -0.0734, -0.0817,  0.5799],
         [ 0.5904, -0.2582,  0.2694, -0.2678, -0.0801],
         [-0.1434, -0.7269, -0.7748,  1.5109,  0.2760],
         [-1.3234,  0.1135,  1.8447,  1.3900,  1.1048],
         [-0.4743,  0.3586,  0.6237,  0.0995, -1.7103],
         [-1.0282, -0.1217,  1.4803, -2.6635,  0.1207],
         [-1.0878, -0.1865, -0.5548, -1.6343,  0.2949],
         [ 0.8009,  1.0363,  0.1929,  0.3628, -0.1960]]),
 tensor([ -1.9926, -11.5465, -14.1181,  10.7254,  -3.6243,   8.0234,  -6.7252,
          28.0304])]

When applicable, it can decode the raw model outputs and compute activations. For instance, let's decode logits to class label indices and binary classes.

In [ ]:
comb_loss.decodes([output1, output2])

[tensor([4, 0, 3, 2, 2, 2, 4, 1]),
 tensor([False, False, False,  True, False,  True, False,  True])]

Similary, here are the activations for each model output.

In [ ]:
comb_loss.activation([output1, output2])

[tensor([[0.1539, 0.2458, 0.1534, 0.1521, 0.2948],
         [0.3238, 0.1386, 0.2348, 0.1372, 0.1656],
         [0.1131, 0.0631, 0.0602, 0.5915, 0.1721],
         [0.0181, 0.0760, 0.4290, 0.2723, 0.2047],
         [0.1196, 0.2750, 0.3585, 0.2122, 0.0347],
         [0.0523, 0.1295, 0.6429, 0.0102, 0.1651],
         [0.1028, 0.2531, 0.1751, 0.0595, 0.4096],
         [0.2615, 0.3309, 0.1424, 0.1687, 0.0965]]),
 tensor([1.1998e-01, 9.6696e-06, 7.3891e-07, 9.9998e-01, 2.5975e-02, 9.9967e-01,
         1.1989e-03, 1.0000e+00])]